In [1]:
import time
import requests
import random
import string
from dataclasses import dataclass, field
import json

import pandas as pd

import AlphaVantage
from AlphaVantage import AlphaVantage as AV

In [2]:
class Ticker:
    def __init__(self, symbol):
        self.symbol = symbol
        self.__items__ = ['BALANCE_SHEET','CASH_FLOW', 'EARNINGS', 'INCOME_STATEMENT', 'OVERVIEW', 'TIME_SERIES_INTRADAY_EXTENDED']
        self.filepath = f'data/master_ticker.h5'         
    
    def __fill_subitem__(self, item):
        targets = {'BALANCE_SHEET': 'quarterlyReports',
                   'CASH_FLOW': 'quarterlyReports', 
                   'EARNINGS': 'quarterlyEarnings', 
                   'INCOME_STATEMENT': 'quarterlyReports', 
                   'OVERVIEW': 'quarterlyReports', 
                   'TIME_SERIES_INTRADAY_EXTENDED': 'timeSeries'}
        return targets[item]
    
    def get(self, item):
        if item.upper() in self.__items__:
            return AV({'symbol': self.symbol, 'function': item.upper()}).request()
        
    def write(self, item, subitem=None):
        item = item.upper()
        if item in self.__items__:
            file = pd.HDFStore(self.filepath, 'a')
            if subitem is None:
                subitem = self.__fill_subitem__(item)
            
            resp = self.get(item)
            if type(resp) == tuple:
                df = pd.DataFrame(resp[1], columns = resp[0])
            elif type(resp) == dict and item != 'OVERVIEW':
                df = pd.DataFrame(resp[subitem])
            elif type(resp) == dict and item == 'OVERVIEW':
                df = pd.Series(resp)
            file.put(f'/{self.symbol}/{item.lower()}', df)
            file.close()
                
    def read(self, item):
        item = item.lower()
        if item.upper() in self.__items__:
            while getattr(self, item, None) is None: 
                try: 
                    df = pd.read_hdf(self.filepath, f'{self.symbol}/{item}')
                    setattr(self, item, df)
                except:
                    print(f'No {item} data stored locally for {self.symbol}.')
                    self.write(item)                    
        
    def run(self):
        for item in self.__items__:
            self.write(item)
            self.read(item)
    
    def __repr__(self):        
        for item in self.__items__:
            self.read(item)
        descr = self.overview.pop('Description')
        return f'{descr}\n{self.overview}'.format()
        

In [3]:
tsla = Ticker('TSLA')

In [4]:
tsla

No cash_flow data stored locally for TSLA.
Request successful.
No earnings data stored locally for TSLA.
Request successful.
No income_statement data stored locally for TSLA.
Request successful.
No time_series_intraday_extended data stored locally for TSLA.
Request successful.


Tesla, Inc. is an American electric vehicle and clean energy company based in Palo Alto, California. Tesla's current products include electric cars, battery energy storage from home to grid-scale, solar panels and solar roof tiles, as well as other related products and services. In 2020, Tesla had the highest sales in the plug-in and battery electric passenger car segments, capturing 16% of the plug-in market (which includes plug-in hybrids) and 23% of the battery-electric (purely electric) market. Through its subsidiary Tesla Energy, the company develops and is a major installer of solar photovoltaic energy generation systems in the United States. Tesla Energy is also one of the largest global suppliers of battery energy storage systems, with 3 GWh of battery storage supplied in 2020.
Symbol                                                         TSLA
AssetType                                              Common Stock
Name                                                      Tesla Inc

In [15]:
tsla = Ticker('TSLA')
tsla.overview.request()
tsla.income_statement.request()
tsla.balance_sheet.request()
tsla.cash_flow.request()
tsla.earnings.request()
tsla.recent_pricing.request()
# pd.DataFrame(tsla.overview.response.json())
# tsla

Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.


In [10]:
'Note' in tsla.overview.response.iter_content()

False

In [29]:
# pd.DataFrame().from_dict(
pd.DataFrame(tsla.balance_sheet.response.json()['quarterlyReports'])
                         # ['quarterlyReports'])

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2021-12-31,USD,62131000000,27100000000,17576000000,17707000000,5757000000,1924000000,25281000000,18884000000,...,1088000000,None,4841000000,294000000,3546000000,30189000000,None,331000000,1000000,1033000000
1,2021-09-30,USD,57834000000,25002000000,16065000000,16095000000,5199000000,1973000000,21886000000,17298000000,...,1716000000,None,6137000000,301000000,3486000000,27053000000,None,-1990000000,1000000,1004000000
2,2021-06-30,USD,55146000000,24693000000,16229000000,16229000000,4733000000,2140000000,19781000000,15665000000,...,1082000000,None,7512000000,292000000,3336000000,24804000000,None,-3608000000,1000000,984000000
3,2021-03-31,USD,52972000000,24705000000,17141000000,17141000000,4132000000,1900000000,17884000000,13868000000,...,1415000000,None,9030000000,273000000,3283000000,23017000000,None,-4750000000,1000000,963000000
4,2020-12-31,USD,52148000000,26717000000,19384000000,19384000000,4101000000,1895000000,18134000000,12747000000,...,2132000000,None,9846000000,264000000,3330000000,22225000000,None,-5399000000,1000000,960000000
5,2020-09-30,USD,45691000000,21744000000,14531000000,14531000000,4218000000,1766000000,15478000000,11848000000,...,2766000000,None,11834000000,173000000,3049000000,16031000000,None,-5669000000,1000000,948000000
6,2020-06-30,USD,38135000000,15336000000,8615000000,8615000000,4018000000,1495000000,14845000000,11009000000,...,233000000,None,11969000000,179000000,2870000000,9855000000,None,-6000000000,0,186000000
7,2020-03-31,USD,37250000000,14893000000,8080000000,8080000000,4494000000,1283000000,14356000000,10638000000,...,229000000,None,11735000000,184000000,2667000000,9173000000,None,-6104000000,0,185000000
8,2019-12-31,USD,34309000000,12103000000,6268000000,6268000000,3552000000,1324000000,15251000000,10396000000,...,228000000,None,11187000000,260000000,2691000000,6618000000,None,-6083000000,1000000,181000000
9,2019-09-30,USD,32795000000,10940000000,5338000000,5338000000,3581000000,1128000000,15035000000,10190000000,...,223000000,None,11129000000,2938000000,2676000000,6040000000,None,-6188000000,0,180000000


In [22]:
pd.DataFrame(tsla.cash_flow.response.json()['quarterlyReports'])

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome
0,2021-12-31,USD,4585000000,None,None,2384000000,-5755000000,997000000,1810000000,-278000000,...,None,None,0,None,None,262000000,None,1412000000,None,2321000000
1,2021-09-30,USD,3147000000,None,None,1545000000,964000000,496000000,1819000000,-135000000,...,None,None,0,0,0,192000000,None,2154000000,None,1618000000
2,2021-06-30,USD,2124000000,None,None,1061000000,1831000000,462000000,1505000000,307000000,...,None,None,0,0,0,70000000,None,1639000000,None,1142000000
3,2021-03-31,USD,1641000000,None,None,857000000,-441000000,426000000,2848000000,-24000000,...,None,None,0,0,0,183000000,None,-1957000000,None,438000000
4,2020-12-31,USD,3019000000,None,None,1774000000,122000000,875000000,1156000000,102000000,...,None,None,4987000000,0,0,5043000000,None,2066000000,None,270000000
5,2020-09-30,USD,2400000000,None,None,1424000000,459000000,455000000,1010000000,314000000,...,None,None,4973000000,0,0,5117000000,None,772000000,None,331000000
6,2020-06-30,USD,964000000,None,None,-51000000,-21000000,402000000,546000000,222000000,...,None,None,0,0,0,57000000,None,485000000,None,104000000
7,2020-03-31,USD,-440000000,None,None,-229000000,1109000000,417000000,455000000,14000000,...,None,None,2309000000,0,0,2469000000,None,1788000000,None,16000000
8,2019-12-31,USD,1425000000,671000000,None,654000000,144000000,566000000,412000000,217000000,...,None,None,0,0,0,96000000,None,1391000000,None,105000000
9,2019-09-30,USD,756000000,None,None,296631000,207284000,476000000,385058000,-18312000,...,None,None,0,0,-432000,70751000,None,503000000,None,143000000


In [28]:
pd.DataFrame(tsla.earnings.response.json()['quarterlyEarnings'])


,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-12-31,2022-01-26,2.54,2.37,0.17,7.173
1,2021-09-30,2021-10-20,1.86,1.6143,0.2457,15.2202
2,2021-06-30,2021-07-26,1.45,0.9789,0.4711,48.1254
3,2021-03-31,2021-04-26,0.93,0.7787,0.1513,19.4298
4,2020-12-31,2021-01-27,0.8,1.0514,-0.2514,-23.911
5,2020-09-30,2020-10-21,0.76,0.6042,0.1558,25.7862
6,2020-06-30,2020-07-22,0.436,-0.0321,0.4681,1458.2555
7,2020-03-31,2020-04-29,0.2281,-0.0415,0.2696,649.6386
8,2019-12-31,2020-01-29,0.4128,0.3506,0.0622,17.741
9,2019-09-30,2019-10-23,0.372,-0.0463,0.4183,903.4557


In [42]:
class Portfolio:
    def __init__(self, name: str, tickers: list):
        self.name = name
        self.tickers = tickers
        self.data = {}
        
    def call(self, ticker):
        assert ticker in self.tickers, 'Ticker unrecognized. Please input a valid symbol.'
        call = AV({'symbol': ticker, 'function': 'OVERVIEW'})
        call.request()
        return call.response.json()
    
    def summarize(self):
        for ticker in self.tickers:
            self.data[ticker] = self.call(ticker)
            time.sleep(20)
    
    def store(self):
        assert all(t for t in self.tickers in self.data.keys()), self.sumarize()
        df = pd.DataFrame(self.data)
        df.to_csv(f'Summary_{self.name}_{dt.today().strftime("%Y%m%d")}')
    
    def read(self):
        file = next(i for i in os.listdir(os.getcwd()) if f'Summary_{self.name}' in i)
        self.data = pd.read_csv(file)
        

In [43]:
sp500 = pd.read_csv('sp500.csv').Symbol.unique()
summary = Portfolio('SP500', sp500)
summary.summarize()

Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.
Request successful.


ConnectionError: HTTPSConnectionPool(host='www.alphavantage.co', port=443): Max retries exceeded with url: /query?symbol=CME&function=OVERVIEW&apikey=7HMKJ7JTYBUHJVX9 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002D4380B8F40>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [45]:
len(pd.DataFrame(summary.data).columns)

98

In [ ]:
overview = {'function':'OVERVIEW'}

companies = df.loc[df.exchange == 'NYSE'].symbol.unique()
for comp in companies:
    overview['symbol'] = comp
    profile = AlphaVantage(overview)
    profile.execute()